# Basics of Keras
[Keras homepage](https://keras.io/): Keras is a high-level neural networks API, written in Python and capable of running on top of TensorFlow, CNTK, or Theano. It was developed with a focus on enabling fast experimentation. Being able to go from idea to result with the least possible delay is key to doing good research.


You may also consider installing the following optional dependencies:

    cuDNN (recommended if you plan on running Keras on GPU).
    HDF5 and h5py (required if you plan on saving Keras models to disk).
    graphviz and pydot (used by visualization utilities to plot model graphs).



**Installation**
- for python2.x: pip install keras
- for python3.x: pip3 install keras

In [ ]:
#!pip install keras

In [1]:
import keras #importing keras library

ModuleNotFoundError: No module named 'keras'

### Building a network

In [ ]:
import numpy as np
from numpy.random import seed
seed(1) # for reproducibility
from keras.models import Sequential
from keras.layers import Dense, Activation # import linear layer (Dense) and activation

#### There are two ways of building a network in keras:
- Sequential: It allows you to build your network by adding layers one-after-other in a sequence. One drawback of this method is that you can't build networks that share layers.
- Functional API: Here you build a network like a graph. Hence more complex networks can be built.

### Sequential
Read more: https://keras.io/models/sequential/

In [ ]:
#Simple 1 layer network 
model = Sequential()
model.add(Dense(10, input_shape=(32,)))
model.add(Activation('softmax'))

In [ ]:
model.summary() # prints the summary of the network
# Notice in Output Shape "None" is batch dims, 10 is feature dims

### Functional API
Read more: https://keras.io/getting-started/functional-api-guide/

Building the same network using functional API.

In [ ]:
from keras.layers import Input
from keras.models import Model
inp = Input(shape=(32,))
l1 = Dense(10)(inp) # See how the dense layer is pointing to inp
act1 = Activation('softmax')(l1)

model = Model(input=inp,output=act1)

model.summary()

In [ ]:
# Building a 2 layer NN for binary classification:

#Simple 1 layer network 
model = Sequential()
model.add(Dense(16, input_shape=(32,)))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))
model.summary()

#### Training

- For supervised learning, we need (x,y) pairs to train our model. Where x is the input data and y is the corresponding ground truth.

Lets sample x and y from a random distribution.

In [ ]:
# Create a set of random input vectors.
# Both the input feature dimension and the input shape of the network should be consistent. Else you will get an error.
x_train = np.random.rand(1000,32)
y_train = np.random.binomial(1, 0.5, 1000) #Sampling from binomial distribution


# Lets check our input and outputs

print("x_train[:5]",x_train[:5])
print("y_train[:5]",y_train[:5])

Similarly we will create our validation and test set

In [ ]:
# Validation Set
x_val = np.random.rand(250,32)
y_val = np.random.binomial(1, 0.5, 250)

# Test Set
x_test = np.random.rand(250,32)
y_test = np.random.binomial(1, 0.5, 250)

Now we will set other hyperparameter and compile the model.

In [ ]:
nb_batch = 32 # batch_size
nb_epoch = 100 # no. of epochs
# Compile model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Check keras documentation for other optimizers
# Since the task here is classification, categorical_crossentropy loss will be used.


We can add some callbacks to the model. This will allow us to eg. save the training/validation accuracy and loss at each epoch ( allows you to detect overfitting!), save the model, implement early stopping, ...

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint('model.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
csvLogger = keras.callbacks.CSVLogger("training_log.csv")
earlyStopping = keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)

callbacks_list = [checkpoint, csvLogger,earlyStopping]


In [ ]:
model.fit(x=x_train, y=y_train, batch_size=nb_batch, epochs=nb_epoch,callbacks=callbacks_list verbose=1, validation_data=(x_val,y_val), shuffle=True)
# Keep shuffle True while training. Why?

#### Questions:
- Why the training loss is decreasing? why validation loss is increasing?
- Why the training accuracy is increasing? why the validation accuracy is almost constant?


In [ ]:
# Testing

test_loss, test_accuracy = model.evaluate(x=x_test,y=y_test,batch_size=8)
print("\n")
print("test_loss:",test_loss,"    test_accuracy:", test_accuracy)

## Convolutional Neural Network


A convolutional neural network (or CNN) is a type of neural network comprises of typically following building blocks:
- Convolutional Layer: These are a set of kernels/filters that convolve with a signal (1D: audio,EEG, etc; 2D: Images; 3D: Videos) to find particular patterns in it based on the kernel type. The kernels or filters are learnable through gradient descent.
- Non-linearity: Relu, Sigmoid, tanh, etc.
- Pooling layer: Downsamples the input signal, which also reduced the necessity to have a larger convolutional layer at the output. It also introduces small translation invariance to the input signal.
- Fully connected layer/Linear layer: They are mainly used to model the actual decision process. Example: classifier.

Hence, in contrast to classical methods where features are handcrafted and then we train a classifier on those features. CNN does both learning features and classification.

![Basic CNN block](cnn_architecture.svg)
Image source: https://developers.google.com/machine-learning/practica/image-classification/convolutional-neural-networks